In [1]:
from jetcam.csi_camera import CSICamera
camera = CSICamera(width=500, height=500, capture_fps=65)
camera.running = True

In [2]:
#!/usr/bin/env python
from flask import Flask, render_template, Response
import cv2
import threading


app = Flask(__name__)

server_running = True
# After gen is called infinite loop will take frame by frame from camera class,
# and yeild it to the client in jpeg data form
def gen():
    """Video streaming generator function."""
    while server_running:
        frame = camera.value
        if frame is not None:
            # Resize the frame to be twice as big
            # frame = cv2.resize(frame, (frame.shape[1] * 2, frame.shape[0] * 2))
            
            # Encode the resized frame into JPEG format
            frame_bytes = cv2.imencode('.jpg', frame)[1].tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')
        else:
            print("Error: Empty frame received.")

# When client is forwarded to /video_feed, it will call 'gen' function
# with imported camera class as a paremeter
@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(gen(), mimetype='multipart/x-mixed-replace; boundary=frame')

# When client demands root, index.html will forward it to video feed
@app.route('/')
def index():
    """Seurveillance camera monitoring"""
    return render_template('index.html')

@app.route('/one')
def hello_one():
    return 'hello one'

@app.route('/image')
def image():
    frame = camera.value
    if frame is not None:
        frame_bytes = cv2.imencode('.jpg', frame)[1].tobytes()
        return Response(frame_bytes, mimetype='image/jpeg')
    else:
        # Handle case when the frame is empty
        return "Error: Empty frame received."

def run_flask():
    app.run(host='0.0.0.0', port=5000, threaded=True, debug=False, use_reloader=False)

if __name__ == '__main__':
    flask_thread = threading.Thread(target=run_flask)
    flask_thread.start()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.12:5000/ (Press CTRL+C to quit)
192.168.1.13 - - [05/Apr/2024 08:57:22] "GET / HTTP/1.1" 200 -
192.168.1.13 - - [05/Apr/2024 08:57:22] "GET /video_feed HTTP/1.1" 200 -
192.168.1.13 - - [05/Apr/2024 08:57:22] "GET /favicon.ico HTTP/1.1" 404 -
192.168.1.13 - - [05/Apr/2024 08:59:19] "GET / HTTP/1.1" 200 -
192.168.1.13 - - [05/Apr/2024 08:59:19] "GET /video_feed HTTP/1.1" 200 -
192.168.1.13 - - [05/Apr/2024 08:59:19] "GET /favicon.ico HTTP/1.1" 404 -
192.168.1.13 - - [05/Apr/2024 09:00:09] "GET / HTTP/1.1" 200 -
192.168.1.13 - - [05/Apr/2024 09:00:09] "GET /video_feed HTTP/1.1" 200 -
192.168.1.13 - - [05/Apr/2024 09:00:09] "GET /favicon.ico HTTP/1.1" 404 -
192.168.1.13 - - [05/Apr/2024 09:00:14] "GET / HTTP/1.1" 200 -
192.168.1.13 - - [05/Apr/2024 09:00:14] "GET /video_feed HTTP/1.1" 200 -
192.168.1.13 - - [05/Apr/2024 09:00:14] "GET /favicon.ico HTTP/1.1" 404 -
192.168.1.13 - - [05/Apr/2024 09:00:48] "GET / HTTP/1.1" 200 -
192.1

In [7]:
server_running = False

In [4]:
camera.running = False
camera.cap.release()

In [ ]:
# lsof -i :5000 
# sudo kill -9 12930